# *Puxando dados fundamentalistas da CVM*

In [4]:
import pandas as pd
import requests
import os
import zipfile
from datetime import datetime

In [5]:
ano_atual = datetime.now().year

anos = range(2010, ano_atual + 1)

url_base = f"http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/"

os.chdir("/Users/wirysmendes/Códigos/ferramentas_financeiras/dados_cvm")

In [6]:
for ano in anos:
        
    download = requests.get(url_base + f"DFP/DADOS/dfp_cia_aberta_{ano}.zip")

    open(f"dfp_cia_aberta_{ano}.zip", "wb").write(download.content)

In [7]:
lista_demonstracoes = []

In [9]:
diretorio_atual = os.getcwd()

lista_demonstracoes = []

for arquivo in os.listdir(diretorio_atual):

    if arquivo.endswith(".zip"):

        caminho_zip = os.path.join(diretorio_atual, arquivo)
        
        try:
            arquivo_zip = zipfile.ZipFile(caminho_zip)

        except zipfile.BadZipFile:

            print(f"{arquivo} não é um zip válido ou está corrompido")

            continue
        
        tipo = "dfp"

        ano = arquivo[-8:-4]
        
        for planilha in arquivo_zip.namelist():

            if planilha != f"{tipo}_cia_aberta_{ano}.csv":

                demonstracao = pd.read_csv(

                    arquivo_zip.open(planilha), 

                    sep=";", decimal=",", encoding="ISO-8859-1", chunksize=1000)
                
                demonstracao = pd.concat(demonstracao, ignore_index=True)

                demonstracao["tipo_doc"] = len(demonstracao) * [tipo]
                
                lista_demonstracoes.append(demonstracao)


In [10]:
base_dados = pd.concat(lista_demonstracoes)

base_dados

/var/folders/sj/grpf2mg12b3813fqgkd99x3c0000gn/T/ipykernel_49347/3483829823.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  base_dados = pd.concat(lista_demonstracoes)


,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,...,TP_RELAT_AUD,TP_PARECER_DECL,NUM_ITEM_PARECER_DECL,TXT_PARECER_DECL,QT_ACAO_ORDIN_CAP_INTEGR,QT_ACAO_PREF_CAP_INTEGR,QT_ACAO_TOTAL_CAP_INTEGR,QT_ACAO_ORDIN_TESOURO,QT_ACAO_PREF_TESOURO,QT_ACAO_TOTAL_TESOURO
0,00.000.000/0001-91,2015-12-31,3,BCO BRASIL S.A.,1023.0,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2014-12-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00.000.000/0001-91,2015-12-31,3,BCO BRASIL S.A.,1023.0,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2015-12-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00.000.000/0001-91,2015-12-31,3,BCO BRASIL S.A.,1023.0,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2014-12-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00.000.000/0001-91,2015-12-31,3,BCO BRASIL S.A.,1023.0,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2015-12-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00.000.000/0001-91,2015-12-31,3,BCO BRASIL S.A.,1023.0,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2014-12-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3527,97.837.181/0001-47,2018-12-31,1,DEXCO S.A.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Parecer do Conselho Fiscal ou Órgão Equivalente,1.0,DURATEX S.A.CNPJ. 97.837.181/0001-47 Companhia...,NaN,NaN,NaN,NaN,NaN,NaN
3528,97.837.181/0001-47,2018-12-31,1,DEXCO S.A.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"Parecer ou Relatório Resumido, se houver, do C...",1.0,DURATEX S.A.CNPJ. 97.837.181/0001-47 Companhia...,NaN,NaN,NaN,NaN,NaN,NaN
3529,97.837.181/0001-47,2018-12-31,1,DEXCO S.A.,NaN,NaN,NaN,NaN,NaN,NaN,...,Sem Ressalva,Relatório do Auditor Independente,1.0,RELATÓRIO DO AUDITOR INDEPENDENTE SOBRE AS DEM...,NaN,NaN,NaN,NaN,NaN,NaN
3530,97.837.181/0001-47,2018-12-31,1,DEXCO S.A.,NaN,NaN,NaN,NaN,NaN,NaN,...,Sem Ressalva,Relatório do Auditor Independente,2.0,"ivas 2, 3 e 11 às demonstrações financeiras.Ba...",NaN,NaN,NaN,NaN,NaN,NaN


# *Tratamento da base*

In [11]:
base_dados[["con_ind", "tipo_dem"]] = base_dados['GRUPO_DFP'].str.split("-", expand = True)

base_dados["tipo_dem"] = base_dados["tipo_dem"].str.strip()
base_dados["con_ind"] = base_dados["con_ind"].str.strip()
base_dados["con_ind"] = base_dados["con_ind"].astype(str)

base_dados

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,...,NUM_ITEM_PARECER_DECL,TXT_PARECER_DECL,QT_ACAO_ORDIN_CAP_INTEGR,QT_ACAO_PREF_CAP_INTEGR,QT_ACAO_TOTAL_CAP_INTEGR,QT_ACAO_ORDIN_TESOURO,QT_ACAO_PREF_TESOURO,QT_ACAO_TOTAL_TESOURO,con_ind,tipo_dem
0,00.000.000/0001-91,2015-12-31,3,BCO BRASIL S.A.,1023.0,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2014-12-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DF Consolidado,Balanço Patrimonial Ativo
1,00.000.000/0001-91,2015-12-31,3,BCO BRASIL S.A.,1023.0,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2015-12-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DF Consolidado,Balanço Patrimonial Ativo
2,00.000.000/0001-91,2015-12-31,3,BCO BRASIL S.A.,1023.0,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2014-12-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DF Consolidado,Balanço Patrimonial Ativo
3,00.000.000/0001-91,2015-12-31,3,BCO BRASIL S.A.,1023.0,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2015-12-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DF Consolidado,Balanço Patrimonial Ativo
4,00.000.000/0001-91,2015-12-31,3,BCO BRASIL S.A.,1023.0,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2014-12-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DF Consolidado,Balanço Patrimonial Ativo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3527,97.837.181/0001-47,2018-12-31,1,DEXCO S.A.,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,DURATEX S.A.CNPJ. 97.837.181/0001-47 Companhia...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN
3528,97.837.181/0001-47,2018-12-31,1,DEXCO S.A.,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,DURATEX S.A.CNPJ. 97.837.181/0001-47 Companhia...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN
3529,97.837.181/0001-47,2018-12-31,1,DEXCO S.A.,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,RELATÓRIO DO AUDITOR INDEPENDENTE SOBRE AS DEM...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN
3530,97.837.181/0001-47,2018-12-31,1,DEXCO S.A.,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,"ivas 2, 3 e 11 às demonstrações financeiras.Ba...",NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN


In [12]:
base_dados = base_dados.loc[:, ~base_dados.columns.isin(["ST_CONTA_FIXA", "COLUNA_DF", "GRUPO_DFP", "NUM_ITEM_PARECER_DECL", "TXT_PARECER_DECL", "QT_ACAO_ORDIN_CAP_INTEGR",
                                                         "QT_ACAO_PREF_CAP_INTEGR", "QT_ACAO_TOTAL_CAP_INTEGR", "QT_ACAO_ORDIN_TESOURO", "QT_ACAO_PREF_TESOURO",
                                                         "QT_ACAO_TOTAL_TESOURO", "TP_RELAT_AUD","TP_PARECER_DECL"])]

In [13]:
base_dados = base_dados[base_dados["ORDEM_EXERC"] != "PENÚLTIMO"]

In [14]:
# verificar tipos de demostracoes disponiveis

lista_dem = base_dados['tipo_dem'].unique()

lista_dem

array(['Balanço Patrimonial Ativo', 'Balanço Patrimonial Passivo',
       'Demonstração do Fluxo de Caixa (Método Direto)',
       'Demonstração do Fluxo de Caixa (Método Indireto)',
       'Demonstração das Mutações do Patrimônio Líquido',
       'Demonstração de Resultado Abrangente',
       'Demonstração do Resultado', 'Demonstração de Valor Adicionado',
       nan], dtype=object)

In [15]:
# verificar nomes das empresas disponiveis

lista_empresas = base_dados['DENOM_CIA'].unique()

lista_empresas

array(['BCO BRASIL S.A.', 'BRB BANCO DE BRASILIA S.A.',
       'CENTRAIS ELET BRAS S.A. - ELETROBRAS', ...,
       'EPR VIA MINEIRA S.A.', 'IGUÁ SERGIPE S.A.', 'SLOT URBANISMO S/A'],
      dtype=object)

# *Exemplo de uso*

In [16]:
weg_dre = base_dados[(base_dados["DENOM_CIA"] == "WEG S.A.") & 
                     (base_dados["tipo_dem"] == "Demonstração do Resultado") &
                    (base_dados["tipo_doc"] == "dfp") & 
                    (base_dados["DS_CONTA"] == "Receita de Venda de Bens e/ou Serviços") & 
                    (base_dados["con_ind"] == "DF Consolidado")] 
                                                         

weg_dre

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,tipo_doc,DT_INI_EXERC,TP_RELAT_AUD,TP_PARECER_DECL,con_ind,tipo_dem
22410,84.429.695/0001-11,2015-12-31,1,WEG S.A.,5410.0,REAL,MIL,ÚLTIMO,2015-12-31,3.01,Receita de Venda de Bens e/ou Serviços,9760323.0000000000,dfp,2015-01-01,NaN,NaN,DF Consolidado,Demonstração do Resultado
22612,84.429.695/0001-11,2014-12-31,1,WEG S.A.,5410.0,REAL,MIL,ÚLTIMO,2014-12-31,3.01,Receita de Venda de Bens e/ou Serviços,7840757.0000000000,dfp,2014-01-01,NaN,NaN,DF Consolidado,Demonstração do Resultado
22497,84.429.695/0001-11,2016-12-31,1,WEG S.A.,5410.0,REAL,MIL,ÚLTIMO,2016-12-31,3.01,Receita de Venda de Bens e/ou Serviços,9367008.0000000000,dfp,2016-01-01,NaN,NaN,DF Consolidado,Demonstração do Resultado
22381,84.429.695/0001-11,2017-12-31,1,WEG S.A.,5410.0,REAL,MIL,ÚLTIMO,2017-12-31,3.01,Receita de Venda de Bens e/ou Serviços,9523830.0000000000,dfp,2017-01-01,NaN,NaN,DF Consolidado,Demonstração do Resultado
22975,84.429.695/0001-11,2013-12-31,1,WEG S.A.,5410.0,REAL,MIL,ÚLTIMO,2013-12-31,3.01,Receita de Venda de Bens e/ou Serviços,6828896.0000000000,dfp,2013-01-01,NaN,NaN,DF Consolidado,Demonstração do Resultado
23379,84.429.695/0001-11,2012-12-31,1,WEG S.A.,5410.0,REAL,MIL,ÚLTIMO,2012-12-31,3.01,Receita de Venda de Bens e/ou Serviços,6173878.0000000000,dfp,2012-01-01,NaN,NaN,DF Consolidado,Demonstração do Resultado
22814,84.429.695/0001-11,2010-12-31,1,WEG S.A.,5410.0,REAL,MIL,ÚLTIMO,2010-12-31,3.01,Receita de Venda de Bens e/ou Serviços,4391973.0000000000,dfp,2010-01-01,NaN,NaN,DF Consolidado,Demonstração do Resultado
23721,84.429.695/0001-11,2011-12-31,1,WEG S.A.,5410.0,REAL,MIL,ÚLTIMO,2011-12-31,3.01,Receita de Venda de Bens e/ou Serviços,5189409.0000000000,dfp,2011-01-01,NaN,NaN,DF Consolidado,Demonstração do Resultado
28547,84.429.695/0001-11,2020-12-31,1,WEG S.A.,5410.0,REAL,MIL,ÚLTIMO,2020-12-31,3.01,Receita de Venda de Bens e/ou Serviços,17469557.0000000000,dfp,2020-01-01,NaN,NaN,DF Consolidado,Demonstração do Resultado
29915,84.429.695/0001-11,2021-12-31,1,WEG S.A.,5410.0,REAL,MIL,ÚLTIMO,2021-12-31,3.01,Receita de Venda de Bens e/ou Serviços,23563338.0000000000,dfp,2021-01-01,NaN,NaN,DF Consolidado,Demonstração do Resultado
